### Introduction

In [1]:
import openvino as ov

In [2]:
#!git clone https://github.com/myshell-ai/OpenVoice

In [3]:
# !mkdir -p OpenVoice/checkpoints/converter/
# !mkdir -p OpenVoice/checkpoints/base_speakers/EN/

# !wget https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/converter/checkpoint.pth -O checkpoints/converter/checkpoint.pth
# !wget https://huggingface.co/myshell-ai/OpenVoice/raw/main/checkpoints/converter/config.json -O checkpoints/converter/config.json

# !wget https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/base_speakers/EN/checkpoint.pth -O checkpoints/base_speakers/EN/checkpoint.pth
# !wget https://huggingface.co/myshell-ai/OpenVoice/raw/main/checkpoints/base_speakers/EN/config.json -O checkpoints/base_speakers/EN/config.json

# !wget https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/base_speakers/EN/en_default_se.pth -O checkpoints/base_speakers/EN/en_default_se.pth

In [4]:
# pip install openvino

# todo: unfreeze dependencies
# %pip install -q --extra-index-url https://download.pytorch.org/whl/cpu  "torch>=2.1.0" "torchaudio>=2.1.0"
# wavmark==0.0.2 also installs torch

In [5]:
# %pip install librosa==0.9.1 \
# faster-whisper==0.9.0 \
# pydub==0.25.1 \
# whisper-timestamped==1.14.2 \
# tqdm \
# inflect==7.0.0 \
# unidecode==1.3.7 \
# eng_to_ipa==0.0.2 \
# wavmark==0.0.2 \
# pypinyin==0.50.0 \
# cn2an==0.5.22 \
# jieba==0.42.1 \
# langid==1.1.6
# gradio==3.48.0 \

In [6]:
%cd OpenVoice

/home/epavel/devel/openvino_notebooks/notebooks/408-openvoice/OpenVoice


/home/epavel/devel/openvino_notebooks/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
import os
import torch
import se_extractor
from api import BaseSpeakerTTS, ToneColorConverter

/home/epavel/devel/openvino_notebooks/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [8]:
ckpt_base = 'checkpoints/base_speakers/EN'
ckpt_converter = 'checkpoints/converter'
device="cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs'

base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

Loaded checkpoint 'checkpoints/base_speakers/EN/checkpoint.pth'
missing/unexpected keys: [] []
Loaded checkpoint 'checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []


In [9]:
source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)

# need to install ffmpeg in the system
reference_speaker = 'resources/example_reference.mp3'
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir='processed', vad=True)

In [11]:
import torch
stn_tst = base_speaker_tts.get_text('this is original text', base_speaker_tts.hps, False)
x_tst = stn_tst.unsqueeze(0)
x_tst_lengths = torch.LongTensor([stn_tst.size(0)])

speed = 1.0
kwargs = dict(noise_scale = 0.667, length_scale = 1.0 / speed, noise_scale_w = 0.6, sdp_ratio = 0.2)

def tts_forward_wrapper(x, x_lengths, sid):
    return base_speaker_tts.model.infer(x, x_lengths, sid,
                                        noise_scale=kwargs['noise_scale'], 
                                        length_scale=kwargs['length_scale'], 
                                        noise_scale_w=kwargs['noise_scale_w'], 
                                        sdp_ratio=kwargs['sdp_ratio'])

y = torch.randn([1, 513, 238], dtype=torch.float32)
y_lengths = torch.LongTensor([y.size(-1)])
tau = 0.3

def converter_forward_wrapper(y, y_lengths, sid_src, sid_tgt):
    return tone_color_converter.model.voice_conversion(y, y_lengths, sid_src, sid_tgt, tau)


import os
core = ov.Core()
ov_tts_fname = 'openvoice_tts.xml'
ov_converter_fname = 'openvoice_converter.xml'

if not os.path.exists(ov_tts_fname):
    for par in base_speaker_tts.model.parameters():
        par.requires_grad = False

    traced_model = torch.jit.trace(tts_forward_wrapper, (x_tst, x_tst_lengths, torch.LongTensor([1])), strict=False);
    ov_tts = ov.convert_model(traced_model)
    ov.save_model(ov_tts, 'openvoice_tts.xml')
else:
    ov_tts = core.read_model(ov_tts_fname)

if not os.path.exists(ov_converter_fname):
    for par in tone_color_converter.model.parameters():
        par.requires_grad = False

    traced_converter = torch.jit.trace(converter_forward_wrapper, (y, y_lengths, source_se, target_se), strict=False)
    ov_converter_model = ov.convert_model(traced_converter)
    ov.save_model(ov_converter_model, 'openvoice_converter.xml')
else:
    ov_converter_model = core.read_model(ov_converter_fname)

compiled_converter = core.compile_model(ov_converter_model, 'CPU')
compiled_tts = core.compile_model(ov_tts, 'CPU')

this is original text.
 length:22
 length:21


### Inference

In [13]:
save_path = f'{output_dir}/output_en_default.wav'

# Run the base speaker tts
text = "This audio is generated by OpenVoice. OpenVoice can achieve multi-lingual voice cloning by simply replace the base speaker." \
""
# "We provide an example with a Chinese base speaker here and we encourage the readers to try demo_part2.ipynb for a detailed demo." \
# "Our online English classes feature lots of useful learning materials and activities to help you develop your reading skills with confidence in a safe and inclusive learning environment." \
# "Practise reading with your classmates in live group classes, get reading support from a personal tutor in one-to-one lessons or practise reading by yourself at your own speed with a self-study course."

src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='default', language='English', speed=1.0, ov_model=compiled_tts)

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message,
    ov_model=compiled_converter)

 > Text splitted to sentences.
This audio is generated by OpenVoice. OpenVoice can achieve multi-lingual voice cloning by simply replace the base speaker.
 > ===========================
ðɪs ˈɑdiˌoʊ ɪz ˈdʒɛnəɹˌeɪtɪd baɪ ˈoʊpən vɔɪs. ˈoʊpən vɔɪs kən əˈtʃiv multi-linguaɫ* vɔɪs ˈkloʊnɪŋ baɪ ˈsɪmpli ˌɹiˈpleɪs ðə beɪs ˈspikəɹ.
 length:136
 length:136


/home/epavel/devel/openvino_notebooks/.venv/lib/python3.10/site-packages/wavmark/models/my_model.py:25: UserWarning: istft will require a complex-valued input tensor in a future PyTorch release. Matching the output from stft with return_complex=True.  (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:978.)
  return torch.istft(signal_wmd_fft, n_fft=self.n_fft, hop_length=self.hop_length, window=window,


In [14]:
import librosa
import librosa.display

# We'll need IPython.display's Audio widget
from IPython.display import Audio

In [17]:
audio, sr = librosa.load(save_path)
Audio(data=audio, rate=sr)

In [21]:
# !pip install ipywebrtc ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 1.0 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 3.7 MB/s eta 0:00:00a 0:00:01m


In [1]:
from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio

camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [6]:
ecorder.audio.value.shapes

NameError: name 'ecorder' is not defined

In [ ]:

Audio(data=sig, rate=sr)